<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/Notebooks/GPTModelKTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from IPython.display import HTML
from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'K80' in s:
  gpu = 'K80'
elif 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
display(HTML(f"<h2>{gpu}</h2>"))
print(s)
# or simply
!nvidia-smi -L

Fri Mar 19 14:20:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    35W / 250W |   6913MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 12.72GB
Available: 9.62GB
Used: 4.29GB
Percentage: 24.3%


In [3]:
import os
import datetime
import time
import random

import pandas as pd
import numpy as np
import re

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from tqdm import tqdm

!pip install transformers

from transformers import (
	AdamW,
	GPT2LMHeadModel,
	GPT2Tokenizer, 
	GPT2Config,
	get_linear_schedule_with_warmup
)

     |████████████████████████████████| 2.0MB 7.3MB/s 
     |████████████████████████████████| 890kB 50.1MB/s 
     |████████████████████████████████| 3.2MB 56.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6a3c7ff7c9d6186afe4095c67daa9e82189e184618e3c5273acd1e16a3c11477
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
!git clone https://github.com/gagan3012/keytotext

Cloning into 'keytotext'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 127 (delta 57), reused 54 (delta 16), pack-reused 0
Receiving objects: 100% (127/127), 2.54 MiB | 10.00 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [5]:
class BOTHData(Dataset):
	def __init__(self, tokenizer, max_source_length,
		max_target_length, type_path):
		super().__init__()
		self.tokenizer = tokenizer
		self.max_source_length = max_source_length
		self.max_target_length = max_target_length
		self.input_ids = []
		self.attn_masks = []
		self._build(type_path)
	
	def __len__(self):
		return len(self.input_ids)
	
	def __getitem__(self, index):

		return self.input_ids[index], self.attn_masks[index]

	def _build(self, type_path):
		if type_path == 'train':
			#df1 = pd.read_csv('/content/keytotext/data/dart_train.csv')
			df = pd.read_csv('/content/keytotext/data/webnlg_train.csv')
		elif type_path == 'eval':
			#df = pd.read_csv('/content/keytotext/data/dart_dev.csv')
			df = pd.read_csv('/content/keytotext/data/webnlg_dev.csv')
		else:
			#df = pd.read_csv('/content/keytotext/data/dart_test.csv')
			df = pd.read_csv('/content/keytotext/data/webnlg_test.csv')
		
	#	df = pd.concat([df1, df2])
		# n = 1 
		# df = df.head(int(len(df)*(n/100)))

		for index, row in df.iterrows():
				line = row['input_text']
				target = row['target_text']
				encodings = self.tokenizer('<|startoftext|>'+ line + ' = ' + target + '<|endoftext|>', truncation=True, max_length=self.max_source_length, padding="max_length")
				self.input_ids.append(torch.tensor(encodings['input_ids']))
				self.attn_masks.append(torch.tensor(encodings['attention_mask']))

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') 

print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.



The max model length is 1024 for this model, although the actual embedding size for GPT small is 768
The beginning of sequence token <|startoftext|> token has the id 50257
The end of sequence token <|endoftext|> has the id 50256
The padding token <|pad|> has the id 50258


In [7]:
batch_size = 4
train_dataset = BOTHData(tokenizer, 512, 512, 'train')
val_dataset = BOTHData(tokenizer, 512, 512, 'val')
test_dataset = BOTHData(tokenizer, 512, 512, 'test')

In [8]:
train_dataloader = DataLoader(
			train_dataset,  # The training samples.
			sampler = RandomSampler(train_dataset), # Select batches randomly
			batch_size = batch_size # Trains with this batch size.
		)

validation_dataloader = DataLoader(
			val_dataset, # The validation samples.
			sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
			batch_size = batch_size # Evaluate with this batch size.
		)
test_dataloader = DataLoader(
			test_dataset, # The validation samples.
			sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
			batch_size = batch_size # Evaluate with this batch size.
		)

In [9]:
configuration = GPT2Config.from_pretrained('distilgpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("distilgpt2", config=configuration)

model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [10]:
device = torch.device("cuda:0")
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [11]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [12]:
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 1000

In [13]:
optimizer = AdamW(model.parameters(),
				  lr = learning_rate,
				  eps = epsilon
				)

In [14]:
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
											num_warmup_steps = warmup_steps, 
											num_training_steps = total_steps)
total_steps

44285

In [15]:
def format_time(elapsed):
	return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [16]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

In [17]:
!pip install wandb

     |████████████████████████████████| 2.0MB 8.0MB/s 
     |████████████████████████████████| 163kB 47.7MB/s 
     |████████████████████████████████| 102kB 12.0MB/s 
     |████████████████████████████████| 133kB 36.2MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=070bbe3e9f69eea56a924ed5591e2235ca4b826b3fc03bc3e4b1ec50ff40a5c4
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=dda0417f5f806519c1f424cc4230657324e3a7c0c799b7fc649752bae3595d6d
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [18]:
%env WANDB_PROJECT = keytotext
import wandb

wandb.init(project="keytotext")

for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                    labels=b_labels,
                    attention_mask=b_masks,
                    token_type_ids=None
                    )

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss
        wandb.log({'Train Loss': batch_loss,'Epoch':epoch_i})
# Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss,
                                                                         elapsed))

            model.eval()

            sample_outputs = model.generate(
                                            bos_token_id=random.randint(1, 30000),
                                            do_sample=True,
                                            top_k=50,
                                            max_length=200,
                                            top_p=0.95,
                                            num_return_sequences=1)
            for i, sample_output in enumerate(sample_outputs):
                print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

# Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

# Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

# ========================================
#               Validation
# ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

# Evaluate data for one epoch
    for batch in tqdm(validation_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids,
                        #                            token_type_ids=None,
                        attention_mask=b_masks,
                        labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss
        wandb.log({'Eval Loss': batch_loss,'Epoch':epoch_i})

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

# Record all statistics from this epoch.
    training_stats.append(
    {
        'epoch': epoch_i + 1,
        'Training Loss': avg_train_loss,
        'Valid. Loss': avg_val_loss,
        'Training Time': training_time,
        'Validation Time': validation_time
    })


env: WANDB_PROJECT=keytotext


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


0it [00:00, ?it/s]


======== Epoch 1 / 5 ========
Training...


...

0:  remembering<H> Abraham A. Ribicoff <R> death place <T> New York City = Abraham A. Ribicoff died in New York City.


8857it [31:31,  4.68it/s]
  0%|          | 2/8857 [00:00<10:26, 14.14it/s]


  Average training loss: 0.04
  Training epoch took: 0:31:32

Running Validation...


100%|██████████| 8857/8857 [10:36<00:00, 13.91it/s]

  Validation Loss: 0.04
  Validation took: 0:10:37


In [19]:
!mkdir model
torch.save(model.state_dict(),'model/pytorch_model.bin')

In [22]:
!cd model
!wget https://s3.amazonaws.com/models.huggingface.co/bert/distilgpt2-config.json
!wget  https://s3.amazonaws.com/models.huggingface.co/bert/distilgpt2-tokenizer.json

--2021-03-19 14:27:03--  https://s3.amazonaws.com/models.huggingface.co/bert/distilgpt2-config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.134.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.134.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 762 [application/json]
Saving to: ‘distilgpt2-config.json’

distilgpt2-config.j 100%[===================>]     762  --.-KB/s    in 0s      

2021-03-19 14:27:03 (20.5 MB/s) - ‘distilgpt2-config.json’ saved [762/762]

--2021-03-19 14:27:03--  https://s3.amazonaws.com/models.huggingface.co/bert/distilgpt2-tokenizer.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.134.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.134.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1355256 (1.3M) [application/json]
Saving to: ‘distilgpt2-tokenizer.json’

distilgpt2-tokenize 100%[===================>]   1.29M  3.35MB/s    in 0.4s    

2021-03-19 14:27:04 (3.

In [23]:
!mv /content/distilgpt2-config.json /content/model/config.json
!mv /content/distilgpt2-tokenizer.json /content/model/tokenizer.json